In [ ]:
%install '.package(path: "$cwd/FastaiNotebook_00_load_data")' FastaiNotebook_00_load_data

In [ ]:
import FastaiNotebook_00_load_data

## Load the data

In [ ]:
import Path
import TensorFlow

In [ ]:
let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath, flat: true)

In [ ]:
import Python

In [ ]:
let np = Python.import("numpy")
let plt = Python.import("matplotlib.pyplot")

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

In [ ]:
let img = np.array(xTrain[0].array.scalars).reshape(28,28)

In [ ]:
plt.figure(figsize: [5,5])
let fig = plt.imshow(X:img, cmap:"gray")
plt.show(fig)

## Matmul

In [ ]:
var weights = Tensor<Float>(randomNormal: [784, 10]) / sqrt(784)
var bias = Tensor(repeating: 0.0, shape: [10])

### From scratch

Flattened version in pure swift

In [ ]:
func swiftMatmul(a:[Float], b:[Float], aShape:[Int], bShape:[Int]) -> [Float]{
    var res:[Float] = Array(repeating: 0.0, count: aShape[0] * bShape[1])
    for i in 0..<aShape[0]{
        for j in 0..<bShape[1]{
            for k in 0..<aShape[1]{
                res[i + aShape[0] * j] += a[i + aShape[0]*k] * b[k + bShape[0]*j]
            }
        }
    }
    return res
}

In [ ]:
let flatA = xTrain[0..<5].array.scalars
let flatB = weights.array.scalars

In [ ]:
let res = swiftMatmul(a: flatA, b:flatB, aShape:[5,784], bShape:[784,10])

In [ ]:
time(repeating: 100) { let _ = swiftMatmul(a: flatA, b:flatB, aShape:[5,784], bShape:[784,10])}

With pointers

In [ ]:
func swiftMatmultPointers(a:[Float], b:[Float], aShape:[Int], bShape:[Int]) -> [Float]{
    var res:[Float] = Array(repeating: 0.0, count: aShape[0] * bShape[1])
    res.withUnsafeMutableBufferPointer { res_ in
        a.withUnsafeBufferPointer { a_ in
            b.withUnsafeBufferPointer { b_ in                              
                for i in 0..<aShape[0]{
                    for j in 0..<bShape[1]{
                        for k in 0..<aShape[1]{
                            res_[i + aShape[0] * j] += a_[i + aShape[0]*k] * b_[k + bShape[0]*j]
                        }
                    }
                }
            }
        }
    }
    return res
}

In [ ]:
time(repeating: 100) { let _ = swiftMatmultPointers(a: flatA, b:flatB, aShape:[5,784], bShape:[784,10])}

With `ShapedArray`

In [ ]:
func shapedArrayMatmul(_ a:ShapedArray<Float>, _ b:ShapedArray<Float>) -> ShapedArray<Float>{
    var res:ShapedArray<Float> = ShapedArray(repeating: 0.0, shape: [a.shape[0], b.shape[1]])
    for i in 0..<a.shape[0]{
        for j in 0..<b.shape[1]{
            for k in 0..<a.shape[1]{
                res[i][j].scalar! += a[i][k].scalar! * b[k][j].scalar!
            }
        }
    }
    return res
}

In [ ]:
let m1 = xTrain[0..<5].reshaped(toShape: [5, 784])
let m2 = weights

In [ ]:
time() { let res = shapedArrayMatmul(m1.array, m2.array)}

With `Tensor`

In [ ]:
func TensorMatmul(_ a:Tensor<Float>, _ b:Tensor<Float>) -> Tensor<Float>{
    var res:Tensor<Float> = Tensor(repeating: 0.0, shape: [a.shape[0], b.shape[1]])
    for i in 0..<a.shape[0]{
        for j in 0..<b.shape[1]{
            for k in 0..<a.shape[1]{
                res[i][j] += a[i][k] * b[k][j]
            }
        }
    }
    return res
}

In [ ]:
time() { let res = TensorMatmul(m1, m2)}

Too long to execture -> Looping over `ShapedArray` or `Tensor` indices is a bad idea!

#### Elementwise ops

Operators (+,-,\*,/) are usually element-wise.

Examples of element-wise operations:

In [ ]:
var a = Tensor([10.0, 6, -4])
var b = Tensor([2.0, 8, 7])
(a,b)

In [ ]:
a + b

Comparison operators (>,<,==,!=,...) are `true` if all the elements of the tensors satisfy the comparison. Elementwise versions have the . prefix: .>, .<, .== ...

In [ ]:
a < b

In [ ]:
a .< b

In [ ]:
var m = Tensor([1.0, 2, 3, 4, 5, 6, 7, 8, 9]).reshaped(to: [3,3])
m

In [ ]:
2 * m

In [ ]:
sqrt((m * m).sum().scalar!)

#### Elementwise matmul

In [ ]:
func elementWiseMatmul(_ a:Tensor<Float>, _ b:Tensor<Float>) -> Tensor<Float>{
    var res = Tensor<Float>(zeros: [a.shape[1], b.shape[0]])
    for i in 0..<a.shape[0]{
        for j in 0..<b.shape[1]{
            res[i][j] = (a[i] * b.slice(lowerBounds: [0,j], upperBounds: [a.shape[1],j+1]).squeezingShape(at: 1)).sum()
        }
    }
    return res
}

In [ ]:
let res = elementWiseMatmul(m1, m2)

In [ ]:
time() { let _ = elementWiseMatmul(m1, m2)}

### Broadcasting

#### Broadcasting with a scalar

In [ ]:
var a = Tensor([10.0, 6.0, -4.0])

In [ ]:
print(a .> 0)

In Tensorflow the operator `>` between tensors will return `true` if all the elements of the first tensor are greater than the ones of the second tensor. `.>` makes the memberwise comparison.

In [ ]:
print(a+1)

In [ ]:
2 * m

#### Broadcasting a vector with a matrix

In [ ]:
let c = Tensor([10.0,20.0,30.0])

By default, broadcasting is done by adding 1 dimensions to the beginning until dimensions of both objects match.

In [ ]:
m + c

In [ ]:
c + m

To broadcast on the other dimensions, one has to use `expandingShape` to add the dimension.

In [ ]:
m + c.expandingShape(at: 1)

In [ ]:
c.expandingShape(at: 1)

#### Matmult with broadcasting

In [ ]:
func broadcastMatmult(_ a:Tensor<Float>, _ b:Tensor<Float>) -> Tensor<Float>{
    var res = Tensor<Float>(zeros: [a.shape[0], b.shape[1]])
    for i in 0..<a.shape[0]{
        res[i] = (a[i].expandingShape(at: 1) * b).sum(squeezingAxes: 0)
    }
    return res
}

In [ ]:
let res = broadcastMatmult(m1, m2)

In [ ]:
time(repeating: 100) { let _ = broadcastMatmult(m1, m2)}

#### Broadcasting rules

In [ ]:
c.expandingShape(at: 0).shape

In [ ]:
c.expandingShape(at: 1).shape

In [ ]:
c.expandingShape(at: 0) * c.expandingShape(at: 1)

In [ ]:
c.expandingShape(at: 0) .> c.expandingShape(at: 1)

### Tensorflow op

In [ ]:
time(repeating: 100) { let _ = matmul(m1, m2)}

### Export

In [ ]:
notebookToScript(fname: (Path.cwd / "01_matmul.ipynb").string)